# 운송부

In [1]:
#client

import socket 
import numpy
import cv2
import dlib


fire_cascade = cv2.CascadeClassifier('fire_detection.xml')
cap = cv2.VideoCapture("http://192.168.0.101:1010/?action=stream")



client_socket = socket.socket(socket.AF_INET,socket.SOCK_STREAM) 
client_socket.connect(('192.168.0.101', 9999))

fire = 'f'
other ='a'
count = 0

while True: 
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    fire = fire_cascade.detectMultiScale(frame, 1.2, 5)

    if len(fire):
        for (x,y,w,h) in fire:
            cv2.rectangle(frame,(x-20,y-20),(x+w+20,y+h+20),(255,0,0),2)
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = frame[y:y+h, x:x+w]
        count += 1
        
        if count>20:
            #client_socket.send(fire.encode('UTF-8')) 
            print('Fire!')
            count = 0
    else:
        count = 0
        #client_socket.send(other.encode('UTF-8'))
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break
cap.release()
cv2.destroyAllWindows()

client_socket.close() 

Fire!
Fire!


error: OpenCV(3.4.2) c:\miniconda3\conda-bld\opencv-suite_1534379934306\work\modules\imgproc\src\color.hpp:253: error: (-215:Assertion failed) VScn::contains(scn) && VDcn::contains(dcn) && VDepth::contains(depth) in function 'cv::CvtHelper<struct cv::Set<3,4,-1>,struct cv::Set<1,-1,-1>,struct cv::Set<0,2,5>,2>::CvtHelper'


In [1]:
import cv2 as cv
import numpy as np
import socket
 
cap = cv.VideoCapture("http://192.168.0.101:1010/?action=stream")
#cap = cv.VideoCapture(0)

client_socket = socket.socket(socket.AF_INET,socket.SOCK_STREAM) 
client_socket.connect(('192.168.0.101', 9999))

# BackgroundSubtractorMOG2
fgbg = cv.createBackgroundSubtractorMOG2()

count = 0
kernel = np.ones((5,5),np.uint8)

fire = 'f'
other = 'a'

while(cap.isOpened()):
    # ret: frame capture 결과(boolean)
    # frame: capture한 frame
    ret, frame = cap.read()
    if (ret):
        frame1 = cv.pyrDown(frame) #엔드 할꺼임 
        fgmask = fgbg.apply(frame) # 엔드 할꺼
        
        vid = cv.medianBlur(fgmask,7) # 미디안 블러로 소금후추 노이즈 제거
        prdown = cv.pyrDown(vid)

        vid1 = cv.bitwise_and(frame1,frame1,mask=prdown) #원본 영상이랑 합치기.

        hsv = cv.cvtColor(vid1,cv.COLOR_BGR2HSV)

        lower = [0, 50, 50]
        upper = [20, 255, 255]

        lower = np.array(lower, dtype="uint8")
        upper = np.array(upper, dtype="uint8")
        mask = cv.inRange(hsv, lower, upper)

        output = cv.bitwise_and(vid1, hsv, mask=mask)
        no_red = cv.countNonZero(mask)

        imgray = cv.cvtColor(output,cv.COLOR_BGR2GRAY)
        ret, thresh = cv.threshold(imgray,127,255,0)
        result = cv.dilate(thresh,kernel,iterations = 3) # 팽창
        
        nlabels, labels, stats, centroids = cv.connectedComponentsWithStats(result)

        for index, centroid in enumerate(centroids):
            if stats[index][0] == 0 and stats[index][1] == 0:
                continue
            if np.any(np.isnan(centroid)):
                continue

        x, y, width, height, area = stats[index]
        centerX, centerY = int(centroid[0]), int(centroid[1])

        if area > 100:
            cv.rectangle(frame1, (x, y), (x + width, y + height), (0, 255, 0),3)

        cv.imshow('frame', frame1)

        if int(no_red) > 1000:
            count += 1
        if count >= 50:
            client_socket.send(fire.encode('UTF-8'))
            count = 0
        else:
            client_socket.send(other.encode('UTF-8'))
        
        k = cv.waitKey(30) & 0xFF
        if k == ord('q'):
            break
    else:
        break
 
cap.release()
cv.destroyAllWindows()



# 1번 모듈

In [2]:
#client

import socket 
import numpy as np
import cv2
import dlib


detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
cap = cv2.VideoCapture("http://192.168.0.102:2020/?action=stream")

ALL = list(range(0, 68)) 
RIGHT_EYEBROW = list(range(17, 22))  
LEFT_EYEBROW = list(range(22, 27))  
RIGHT_EYE = list(range(36, 42))  
LEFT_EYE = list(range(42, 48))  
NOSE = list(range(27, 36))  
MOUTH_OUTLINE = list(range(48, 61))  
MOUTH_INNER = list(range(61, 68)) 
JAWLINE = list(range(0, 17)) 

index = ALL

client_socket = socket.socket(socket.AF_INET,socket.SOCK_STREAM) 
client_socket.connect(('192.168.0.102', 9999))


buzz = 'b'

while True: 
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    dets = detector(gray, 1)
  
    if len(dets):
            for face in dets:
                shape = predictor(frame, face) 
                list_points = []
            cv2.rectangle(frame, (face.left(), face.top()), (face.right(), face.bottom()),
                (0, 0, 255), 3)
            client_socket.send(buzz.encode()) 
    cv2.imshow('result', frame)

    k = cv2.waitKey(1) & 0xff
    if k == 27:
        break
cap.release()
cv2.destroyAllWindows()

client_socket.close() 

# 2번 모듈

In [3]:
#client

import socket 
import numpy as np
import cv2
import dlib


detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
cap = cv2.VideoCapture("http://192.168.0.103:3030/?action=stream")

ALL = list(range(0, 68)) 
RIGHT_EYEBROW = list(range(17, 22))  
LEFT_EYEBROW = list(range(22, 27))  
RIGHT_EYE = list(range(36, 42))  
LEFT_EYE = list(range(42, 48))  
NOSE = list(range(27, 36))  
MOUTH_OUTLINE = list(range(48, 61))  
MOUTH_INNER = list(range(61, 68)) 
JAWLINE = list(range(0, 17)) 

index = ALL

client_socket = socket.socket(socket.AF_INET,socket.SOCK_STREAM) 
client_socket.connect(('192.168.0.103', 9999))


buzz = 'b'

while True: 
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    dets = detector(gray, 1)
  
    if len(dets):
            for face in dets:
                shape = predictor(frame, face) 
                list_points = []
            cv2.rectangle(frame, (face.left(), face.top()), (face.right(), face.bottom()),
                (0, 0, 255), 3)
            client_socket.send(buzz.encode()) 
    cv2.imshow('result', frame)

    k = cv2.waitKey(1) & 0xff
    if k == 27:
        break
cap.release()
cv2.destroyAllWindows()

client_socket.close() 

error: OpenCV(3.4.2) c:\miniconda3\conda-bld\opencv-suite_1534379934306\work\modules\imgproc\src\color.hpp:253: error: (-215:Assertion failed) VScn::contains(scn) && VDcn::contains(dcn) && VDepth::contains(depth) in function 'cv::CvtHelper<struct cv::Set<3,4,-1>,struct cv::Set<1,-1,-1>,struct cv::Set<0,2,5>,2>::CvtHelper'
